In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

Using TensorFlow backend.


In [2]:
def data():
    # Split train & validation set
    FFF_Train_Decile0 = pd.read_pickle("FFF_Train_Decile0.pkl")
    FFF_Train_Targets_1D = pd.read_pickle("FFF_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("FFF_Test_Decile0.pkl")
    y_test = pd.read_pickle("FFF_Test_Targets_1D.pkl")
    
    X_train, X_val, y_train, y_val = train_test_split(FFF_Train_Decile0, FFF_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones',
                moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None,
                beta_constraint=None, gamma_constraint=None))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,75,100,125,150])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM FFF Decile 0')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

3055/3055 [==============================] - 1s 465us/step - loss: 0.6086 - acc: 0.6687 - val_loss: 0.5912 - val_acc: 0.6963
Epoch 35/75
3055/3055 [==============================] - 1s 473us/step - loss: 0.6101 - acc: 0.6704 - val_loss: 0.5911 - val_acc: 0.6963
Epoch 36/75
3055/3055 [==============================] - 1s 484us/step - loss: 0.6103 - acc: 0.6589 - val_loss: 0.5912 - val_acc: 0.6963
Epoch 37/75
3055/3055 [==============================] - 1s 468us/step - loss: 0.6056 - acc: 0.6730 - val_loss: 0.5912 - val_acc: 0.6963
Epoch 38/75
3055/3055 [==============================] - 1s 421us/step - loss: 0.6111 - acc: 0.6655 - val_loss: 0.5910 - val_acc: 0.6963
Epoch 39/75
3055/3055 [==============================] - 2s 527us/step - loss: 0.6076 - acc: 0.6668 - val_loss: 0.5912 - val_acc: 0.6963
Epoch 40/75
3055/3055 [==============================] - 1s 432us/step - loss: 0.6120 - acc: 0.6635 - val_loss: 0.5913 - val_acc: 0.6898
Epoch 41/75
3055/3055 [==============================

3055/3055 [==============================] - 2s 492us/step - loss: 0.6061 - acc: 0.6750 - val_loss: 0.5901 - val_acc: 0.6832

Epoch 00017: ReduceLROnPlateau reducing learning rate to 1e-05.
Epoch 18/75
3055/3055 [==============================] - 1s 447us/step - loss: 0.6070 - acc: 0.6743 - val_loss: 0.5901 - val_acc: 0.6846
Epoch 19/75
3055/3055 [==============================] - 1s 448us/step - loss: 0.6030 - acc: 0.6812 - val_loss: 0.5901 - val_acc: 0.6846
Epoch 20/75
3055/3055 [==============================] - 1s 412us/step - loss: 0.6051 - acc: 0.6740 - val_loss: 0.5901 - val_acc: 0.6846
Epoch 21/75
3055/3055 [==============================] - 1s 442us/step - loss: 0.6042 - acc: 0.6714 - val_loss: 0.5900 - val_acc: 0.6846
Epoch 22/75
3055/3055 [==============================] - 1s 468us/step - loss: 0.6060 - acc: 0.6743 - val_loss: 0.5900 - val_acc: 0.6846
Epoch 23/75
3055/3055 [==============================] - 2s 500us/step - loss: 0.6025 - acc: 0.6809 - val_loss: 0.5900 - val_

3055/3055 [==============================] - 4s 1ms/step - loss: 0.6792 - acc: 0.5843 - val_loss: 0.6366 - val_acc: 0.6859
Epoch 2/100
3055/3055 [==============================] - 2s 496us/step - loss: 0.6254 - acc: 0.6570 - val_loss: 0.5919 - val_acc: 0.6885
Epoch 3/100
3055/3055 [==============================] - 1s 479us/step - loss: 0.6074 - acc: 0.6625 - val_loss: 0.5872 - val_acc: 0.6885
Epoch 4/100
3055/3055 [==============================] - 2s 492us/step - loss: 0.6128 - acc: 0.6655 - val_loss: 0.5881 - val_acc: 0.6885
Epoch 5/100
3055/3055 [==============================] - 1s 449us/step - loss: 0.6093 - acc: 0.6632 - val_loss: 0.5907 - val_acc: 0.6846
Epoch 6/100
3055/3055 [==============================] - 1s 454us/step - loss: 0.6069 - acc: 0.6697 - val_loss: 0.5868 - val_acc: 0.7003
Epoch 7/100
3055/3055 [==============================] - 1s 489us/step - loss: 0.6076 - acc: 0.6697 - val_loss: 0.5878 - val_acc: 0.6963
Epoch 8/100
3055/3055 [==============================] 

3055/3055 [==============================] - 1s 456us/step - loss: 0.6061 - acc: 0.6700 - val_loss: 0.5880 - val_acc: 0.6950
Epoch 60/100
3055/3055 [==============================] - 1s 485us/step - loss: 0.6049 - acc: 0.6655 - val_loss: 0.5879 - val_acc: 0.6963
Epoch 61/100
3055/3055 [==============================] - 1s 460us/step - loss: 0.6062 - acc: 0.6694 - val_loss: 0.5877 - val_acc: 0.6950
Epoch 62/100
3055/3055 [==============================] - 1s 446us/step - loss: 0.6088 - acc: 0.6651 - val_loss: 0.5879 - val_acc: 0.6950
Epoch 63/100
3055/3055 [==============================] - 1s 486us/step - loss: 0.6085 - acc: 0.6671 - val_loss: 0.5880 - val_acc: 0.6937
Epoch 64/100
3055/3055 [==============================] - 1s 489us/step - loss: 0.6045 - acc: 0.6717 - val_loss: 0.5878 - val_acc: 0.6963
Epoch 65/100
3055/3055 [==============================] - 2s 639us/step - loss: 0.6031 - acc: 0.6782 - val_loss: 0.5878 - val_acc: 0.6963
Epoch 66/100
3055/3055 [=======================

...

3055/3055 [==============================] - 1s 337us/step - loss: 0.6077 - acc: 0.6589 - val_loss: 0.5915 - val_acc: 0.6846
Epoch 18/125
3055/3055 [==============================] - 1s 341us/step - loss: 0.6106 - acc: 0.6664 - val_loss: 0.5915 - val_acc: 0.6846

Epoch 00018: ReduceLROnPlateau reducing learning rate to 1e-05.
Epoch 19/125
3055/3055 [==============================] - 1s 348us/step - loss: 0.6066 - acc: 0.6628 - val_loss: 0.5913 - val_acc: 0.6846
Epoch 20/125
3055/3055 [==============================] - 1s 354us/step - loss: 0.6109 - acc: 0.6625 - val_loss: 0.5914 - val_acc: 0.6846
Epoch 21/125
3055/3055 [==============================] - 1s 332us/step - loss: 0.6078 - acc: 0.6671 - val_loss: 0.5915 - val_acc: 0.6846
Epoch 22/125
3055/3055 [==============================] - 1s 334us/step - loss: 0.6149 - acc: 0.6661 - val_loss: 0.5915 - val_acc: 0.6846
Epoch 23/125
3055/3055 [==============================] - 1s 373us/step - loss: 0.6077 - acc: 0.6651 - val_loss: 0.5914 

Epoch 76/125
3055/3055 [==============================] - 1s 339us/step - loss: 0.6094 - acc: 0.6714 - val_loss: 0.5906 - val_acc: 0.6846
Epoch 77/125
3055/3055 [==============================] - 1s 325us/step - loss: 0.6062 - acc: 0.6671 - val_loss: 0.5907 - val_acc: 0.6846
Epoch 78/125
3055/3055 [==============================] - 1s 335us/step - loss: 0.6056 - acc: 0.6625 - val_loss: 0.5906 - val_acc: 0.6846
Epoch 79/125
3055/3055 [==============================] - 1s 308us/step - loss: 0.6079 - acc: 0.6648 - val_loss: 0.5906 - val_acc: 0.6846
Epoch 80/125
3055/3055 [==============================] - 1s 355us/step - loss: 0.6068 - acc: 0.6727 - val_loss: 0.5907 - val_acc: 0.6846
Epoch 81/125
3055/3055 [==============================] - 1s 333us/step - loss: 0.6085 - acc: 0.6651 - val_loss: 0.5907 - val_acc: 0.6846
Epoch 82/125
3055/3055 [==============================] - 1s 346us/step - loss: 0.6079 - acc: 0.6707 - val_loss: 0.5905 - val_acc: 0.6846
Epoch 83/125
3055/3055 [==========

3055/3055 [==============================] - 2s 506us/step - loss: 0.6073 - acc: 0.6704 - val_loss: 0.5906 - val_acc: 0.6793
Epoch 10/100
3055/3055 [==============================] - 1s 467us/step - loss: 0.6069 - acc: 0.6651 - val_loss: 0.5896 - val_acc: 0.6780
Epoch 11/100
3055/3055 [==============================] - 2s 504us/step - loss: 0.6039 - acc: 0.6740 - val_loss: 0.5890 - val_acc: 0.6819
Epoch 12/100
3055/3055 [==============================] - 1s 482us/step - loss: 0.6026 - acc: 0.6750 - val_loss: 0.5890 - val_acc: 0.6832
Epoch 13/100
3055/3055 [==============================] - 2s 502us/step - loss: 0.6041 - acc: 0.6743 - val_loss: 0.5893 - val_acc: 0.6832

Epoch 00013: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
Epoch 14/100
3055/3055 [==============================] - 1s 490us/step - loss: 0.6049 - acc: 0.6756 - val_loss: 0.5893 - val_acc: 0.6819
Epoch 15/100
3055/3055 [==============================] - 1s 466us/step - loss: 0.6030 - acc: 0.6818 - 

3055/3055 [==============================] - 1s 415us/step - loss: 0.6045 - acc: 0.6792 - val_loss: 0.5894 - val_acc: 0.6819
Epoch 68/100
3055/3055 [==============================] - 1s 453us/step - loss: 0.6087 - acc: 0.6651 - val_loss: 0.5893 - val_acc: 0.6806
Epoch 69/100
3055/3055 [==============================] - 1s 439us/step - loss: 0.6043 - acc: 0.6828 - val_loss: 0.5894 - val_acc: 0.6806
Epoch 70/100
3055/3055 [==============================] - 1s 452us/step - loss: 0.6069 - acc: 0.6750 - val_loss: 0.5894 - val_acc: 0.6819
Epoch 71/100
3055/3055 [==============================] - 1s 447us/step - loss: 0.6089 - acc: 0.6720 - val_loss: 0.5894 - val_acc: 0.6819
Epoch 72/100
3055/3055 [==============================] - 1s 489us/step - loss: 0.6062 - acc: 0.6733 - val_loss: 0.5893 - val_acc: 0.6806
Epoch 73/100
3055/3055 [==============================] - 2s 505us/step - loss: 0.5990 - acc: 0.6789 - val_loss: 0.5893 - val_acc: 0.6806
Epoch 74/100
3055/3055 [=======================

3055/3055 [==============================] - 1s 325us/step - loss: 0.6080 - acc: 0.6697 - val_loss: 0.5879 - val_acc: 0.6832
Epoch 26/50
3055/3055 [==============================] - 1s 316us/step - loss: 0.6075 - acc: 0.6759 - val_loss: 0.5878 - val_acc: 0.6846
Epoch 27/50
3055/3055 [==============================] - 1s 330us/step - loss: 0.6080 - acc: 0.6707 - val_loss: 0.5879 - val_acc: 0.6832
Epoch 28/50
3055/3055 [==============================] - 1s 331us/step - loss: 0.6076 - acc: 0.6687 - val_loss: 0.5878 - val_acc: 0.6846
Epoch 29/50
3055/3055 [==============================] - 1s 310us/step - loss: 0.6073 - acc: 0.6759 - val_loss: 0.5878 - val_acc: 0.6846
Epoch 30/50
3055/3055 [==============================] - 1s 322us/step - loss: 0.6067 - acc: 0.6720 - val_loss: 0.5878 - val_acc: 0.6846
Epoch 31/50
3055/3055 [==============================] - 1s 309us/step - loss: 0.6076 - acc: 0.6723 - val_loss: 0.5877 - val_acc: 0.6859
Epoch 32/50
3055/3055 [==============================

In [3]:
0.93484

0.93484